In [1]:
'''
Author: ChenHJ
Date: 2023-01-12 23:39:25
LastEditors: ChenHJ
LastEditTime: 2023-01-12 23:40:01
FilePath: /0302code/obs_CMIP_sIndR_ENSO_related.ipynb
Aim: 
01/13/2023:
用于计算与ENSO-related和ENSO-unrelated的印度半岛南部降水与环流变量之间的关系

01/12/2023:
用于计算ENSO-related的印度半岛南部降水和unrelated的印度半岛南部降水分别对应的降水，环流，在historical、ssp585时期
Mission: 
'''
from mailbox import _PartialFile
import numpy as np
import xarray as xr
import os
import re
from cdo import *
import shutil
import sys

sys.path.append("/home/ys17-23/chenhj/self_def/")
import plot as sepl
import cal as ca
import pandas as pd
from importlib import reload

import metpy.calc as mpcalc
import metpy.constants as constants
import geocat.comp
from windspharm.xarray import VectorWind
import skill_metrics as sm
from brokenaxes import brokenaxes

# sd.path.append("/home/ys17-23/chenhj/1201code/self_def.py")

cdo = Cdo

# for plot
import proplot as pplt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter
from cartopy.mpl.ticker import LatitudeFormatter
from cartopy.util import add_cyclic_point
from matplotlib.ticker import MultipleLocator
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import t
from scipy import signal
from eofs.multivariate.standard import MultivariateEof
from eofs.standard import Eof
import statsmodels.api as sm

# 读取数据

## 观测

In [2]:
fpreGPCP = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/GPCP_r144x72_197901-201412.nc"
)
preGPCP = fpreGPCP["precip"]
preGPCP_JJA = ca.p_time(preGPCP, 6, 8, True)
preGPCP_JJA_detrend = ca.detrend_dim(preGPCP_JJA.copy(), "time", deg=1, demean=False)

# preAIR = xr.open_dataarray("/home/ys17-23/Extension/All_India_Rainfall_index/AIR_mmperday.nc")
# preAIR_JJA = ca.p_time(preAIR, 6, 8, True)
# preAIR_JJA = preAIR_JJA.sel(time=(preAIR_JJA.time.dt.year>=1979) & (preAIR_JJA.time.dt.year <=2014))
# preAIR_JJA = ca.detrend_dim(preAIR_JJA, "time", deg=1, demean=False)

fpreCRU = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/cru_ts4.01_r144x72_195001-201412.nc")
preCRU = fpreCRU["pre"].sel(time=(fpreCRU["time"].dt.year>=1979) & (fpreCRU.time.dt.year<=2014))
preCRU_JJA = ca.p_time(preCRU, 6, 8, True)/30.67
preCRU_JJA.attrs["units"] = "mm/day"
preCRU_JJA = ca.detrend_dim(preCRU_JJA, "time", deg=1, demean=False)

fhgtERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/hgt_mon_r144x72_195001-201412.nc")
hgtERA5 = fhgtERA5["z"].sel(time=fhgtERA5["time"].dt.year>=1979)

fuERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/uwind_mon_r144x72_195001-201412.nc"
)
uERA5 = fuERA5["u"].sel(time=fuERA5["time"].dt.year>=1979)

fvERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/vwind_mon_r144x72_195001-201412.nc"
)
vERA5 = fvERA5["v"].sel(time=fvERA5["time"].dt.year>=1979)

fspERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/sp_mon_r144x72_195001-201412.nc"
)
spERA5 = fspERA5["sp"].sel(time=fspERA5["time"].dt.year>=1979)

fqERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/q_mon_r144x72_195001-201412.nc")
qERA5 = fqERA5["q"].sel(time=fqERA5["time"].dt.year>=1979)

fwERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/omega_mon_r144x72_195001-201412.nc")
wERA5 = fwERA5["w"].sel(time=fwERA5["time"].dt.year>=1979)

fsstHad = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/HadISST_r144x72_1870-2020.nc")
sstHad = fsstHad["sst"].sel(time=(fsstHad["time"].dt.year>=1979) & (fsstHad["time"].dt.year<=2014))

hgtERA5_ver_JJA = ca.p_time(hgtERA5, 6, 8, True)
# hgtERA5_ver_JJA = hgtERA5_ver_JJA-hgtERA5_ver_JJA.mean(dim="lon", skipna=True)
hgtERA5_ver_JJA = ca.detrend_dim(hgtERA5_ver_JJA, "time", deg=1, demean=False)

uERA5_ver_MAM, uERA5_ver_JJA, uERA5_ver_SON, uERA5_ver_DJF = ca.mon_to_season(uERA5)
uERA5_ver_MAM = ca.detrend_dim(uERA5_ver_MAM, "time", deg=1, demean=False)
uERA5_ver_JJA = ca.detrend_dim(uERA5_ver_JJA, "time", deg=1, demean=False)
uERA5_ver_SON = ca.detrend_dim(uERA5_ver_SON, "time", deg=1, demean=False)
uERA5_ver_DJF = ca.detrend_dim(uERA5_ver_DJF, "time", deg=1, demean=False)

vERA5_ver_MAM, vERA5_ver_JJA, vERA5_ver_SON, vERA5_ver_DJF = ca.mon_to_season(vERA5)
vERA5_ver_MAM = ca.detrend_dim(vERA5_ver_MAM, "time", deg=1, demean=False)
vERA5_ver_JJA = ca.detrend_dim(vERA5_ver_JJA, "time", deg=1, demean=False)
vERA5_ver_SON = ca.detrend_dim(vERA5_ver_SON, "time", deg=1, demean=False)
vERA5_ver_DJF = ca.detrend_dim(vERA5_ver_DJF, "time", deg=1, demean=False)

qERA5_ver_JJA = ca.p_time(qERA5, 6, 9, True)
qERA5_ver_JJA = ca.detrend_dim(qERA5_ver_JJA, "time", deg=1, demean=False)

spERA5_MAM, spERA5_JJA, spERA5_SON, spERA5_DJF = ca.mon_to_season(spERA5)
spERA5_MAM = ca.detrend_dim(spERA5_MAM, "time", deg=1, demean=False)
spERA5_JJA = ca.detrend_dim(spERA5_JJA, "time", deg=1, demean=False)
spERA5_SON = ca.detrend_dim(spERA5_SON, "time", deg=1, demean=False)
spERA5_DJF = ca.detrend_dim(spERA5_DJF, "time", deg=1, demean=False)

wERA5_MAM, wERA5_JJA, wERA5_SON, wERA5_DJF = ca.mon_to_season(wERA5)
wERA5_MAM = ca.detrend_dim(wERA5_MAM, "time", deg=1, demean=False)
wERA5_JJA = ca.detrend_dim(wERA5_JJA, "time", deg=1, demean=False)
wERA5_SON = ca.detrend_dim(wERA5_SON, "time", deg=1, demean=False)
wERA5_DJF = ca.detrend_dim(wERA5_DJF, "time", deg=1, demean=False)
sstHad_MAM, sstHad_JJA, sstHad_SON, sstHad_DJF = ca.mon_to_season(sstHad)
sstHad_MAM = ca.detrend_dim(sstHad_MAM, "time", deg=1, demean=False)
sstHad_JJA = ca.detrend_dim(sstHad_JJA, "time", deg=1, demean=False)
sstHad_SON = ca.detrend_dim(sstHad_SON, "time", deg=1, demean=False)
sstHad_DJF = ca.detrend_dim(sstHad_DJF, "time", deg=1, demean=False)
ftERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/temp_mon_r144x72_195001-201412.nc")
tERA5 = ftERA5["t"].sel(time=ftERA5["time"].dt.year>=1979)
tERA5_ver_JJA = ca.p_time(tERA5, 6, 8, True)
tERA5_ver_JJA = ca.detrend_dim(tERA5_ver_JJA, "time", deg=1, demean=False)
divERA5_ver_JJA = ca.cal_divergence(uERA5_ver_JJA.sel(level=[200.0, 850.0]), vERA5_ver_JJA.sel(level=[200.0, 850.0]))
divERA5_ver_JJA = ca.detrend_dim(divERA5_ver_JJA, "time", deg=1, demean=False)

#   calculate the waver vapor vertical intergration
ptop = 1
g = 9.8
ERA5level = qERA5_ver_JJA.coords["level"].where(qERA5_ver_JJA.coords["level"] >= ptop) * 100.0
ERA5level.attrs["units"] = "Pa"
ERA5dp = geocat.comp.dpres_plevel(ERA5level, spERA5_JJA, ptop*100)
ERA5dpg = ERA5dp / g
ERA5dpg.attrs["units"] = "kg/m2"
# calculate the water vapor transport
integration_top = 200
uq_sl_ERA5_JJA = uERA5_ver_JJA.loc[:, integration_top:, :, :] * qERA5_ver_JJA.loc[:, integration_top:, :, :].data * 1000.0
vq_sl_ERA5_JJA = vERA5_ver_JJA.loc[:, integration_top:, :, :] * qERA5_ver_JJA.loc[:, integration_top:, :, :].data * 1000.0
uq_sl_ERA5_JJA.attrs["units"] = "[m/s][g/kg]"
vq_sl_ERA5_JJA.attrs["units"] = "[m/s][g/kg]"
# calculate the whole levels water vapor transport
uqERA5_ver_JJA = (uq_sl_ERA5_JJA * ERA5dpg[:, list(ERA5level).index(integration_top*100):, :, :].data).sum(dim="level") / 1e05
vqERA5_ver_JJA = (vq_sl_ERA5_JJA * ERA5dpg[:, list(ERA5level).index(integration_top*100):, :, :].data).sum(dim="level") / 1e05
uqERA5_ver_JJA.attrs["units"] = "100kg m-1 s-1"
vqERA5_ver_JJA.attrs["units"] = "100kg m-1 s-1"
uqERA5_ver_JJA = ca.detrend_dim(uqERA5_ver_JJA, "time", deg=1, demean=False)
vqERA5_ver_JJA = ca.detrend_dim(vqERA5_ver_JJA, "time", deg=1, demean=False)
# calculate the water vapor transportation divergence
uqvqdivERA5_ver_JJA = ca.cal_divergence(uqERA5_ver_JJA, vqERA5_ver_JJA)
uqvqdivERA5_ver_JJA = ca.detrend_dim(uqvqdivERA5_ver_JJA, "time", deg=1, demean=False)

## historical

In [3]:
fprehis = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/pr_historical_r144x72_197901-201412.nc")
prehis_JJA = fprehis["pr"].sel(time=fprehis["time"].dt.year>=1979)
prehis_JJA.attrs["units"] = "mm/day"
prehis_JJA.attrs["standard_name"] = "precipitation"

fhgthis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/zg_historical_r144x72_197901-201412.nc")
hgthis_ver_JJA = fhgthis_ver_JJA["zg"].sel(time=fhgthis_ver_JJA["time"].dt.year>=1979)
# hgthis_ver_JJA = hgthis_ver_JJA - hgthis_ver_JJA.mean(dim="lon", skipna=True)

fuhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/ua_historical_r144x72_197901-201412.nc")
uhis_ver_JJA = fuhis_ver_JJA["ua"].sel(time=fuhis_ver_JJA["time"].dt.year>=1979)

fvhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/va_historical_r144x72_197901-201412.nc")
vhis_ver_JJA = fvhis_ver_JJA["va"].sel(time=fvhis_ver_JJA["time"].dt.year>=1979)

# fwhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/wap_historical_r144x72_197901-201412.nc") 
# whis_ver_JJA = fwhis_ver_JJA["wap"].sel(time=fwhis_ver_JJA["time"].dt.year>=1979)

fssthis_MAM = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/MAM/detrend/tos_historical_r144x72_197901-201412.nc")
ssthis_MAM = fssthis_MAM["__xarray_dataarray_variable__"].sel(time=fssthis_MAM["time"].dt.year>=1979)

fssthis_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/tos_historical_r144x72_197901-201412.nc")
ssthis_JJA = fssthis_JJA["__xarray_dataarray_variable__"].sel(time=fssthis_JJA["time"].dt.year>=1979)

fssthis_SON = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/SON/detrend/tos_historical_r144x72_197901-201412.nc")
ssthis_SON = fssthis_SON["__xarray_dataarray_variable__"].sel(time=fssthis_SON["time"].dt.year>=1979)

fssthis_DJF = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/DJF/detrend/tos_historical_r144x72_197901-201412.nc")
ssthis_DJF = fssthis_DJF["__xarray_dataarray_variable__"].sel(time=fssthis_DJF["time"].dt.year>=1979)

fuhis_MAM = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/MAM/detrend/u_historical_r144x72_197901-201412.nc")
uhis_MAM = fuhis_MAM["u"].sel(time=fuhis_MAM["time"].dt.year>=1979)

fuhis_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/u_historical_r144x72_197901-201412.nc")
uhis_JJA = fuhis_JJA["u"].sel(time=fuhis_JJA["time"].dt.year>=1979)

fuhis_SON = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/SON/detrend/u_historical_r144x72_197901-201412.nc")
uhis_SON = fuhis_SON["u"].sel(time=fuhis_SON["time"].dt.year>=1979)

fuhis_DJF = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/DJF/detrend/u_historical_r144x72_197901-201412.nc")
uhis_DJF = fuhis_DJF["u"].sel(time=fuhis_DJF["time"].dt.year>=1979)

fvhis_MAM = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/MAM/detrend/v_historical_r144x72_197901-201412.nc")
vhis_MAM = fvhis_MAM["v"].sel(time=fvhis_MAM["time"].dt.year>=1979)

fvhis_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/v_historical_r144x72_197901-201412.nc")
vhis_JJA = fvhis_JJA["v"].sel(time=fvhis_JJA["time"].dt.year>=1979)

fvhis_SON = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/SON/detrend/v_historical_r144x72_197901-201412.nc")
vhis_SON = fvhis_SON["v"].sel(time=fvhis_SON["time"].dt.year>=1979)

fvhis_DJF = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/DJF/detrend/v_historical_r144x72_197901-201412.nc")
vhis_DJF = fvhis_DJF["v"].sel(time=fvhis_DJF["time"].dt.year>=1979)

fthis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/ta_historical_r144x72_197901-201412.nc")
this_ver_JJA = fthis_ver_JJA["ta"].sel(time=fthis_ver_JJA["time"].dt.year>=1979)

sphis_MAM = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/MAM/detrend/sp_historical_r144x72_197901-201412.nc")
sphis_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/sp_historical_r144x72_197901-201412.nc")
sphis_SON = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/SON/detrend/sp_historical_r144x72_197901-201412.nc")
sphis_DJF = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/DJF/detrend/sp_historical_r144x72_197901-201412.nc")

fuqhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/his_uq_dpg.nc")
uqhis_ver_JJA = fuqhis_ver_JJA["uq_dpg"].sel(time=fuqhis_ver_JJA["time"].dt.year>=1979)

fvqhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/his_vq_dpg.nc")
vqhis_ver_JJA = fvqhis_ver_JJA["vq_dpg"].sel(time=fvqhis_ver_JJA["time"].dt.year>=1979)

fuqvqdivhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/his_div_uqvq_dpg.nc")
uqvqdivhis_ver_JJA = fuqvqdivhis_ver_JJA["div_uqvq"].sel(time=fuqvqdivhis_ver_JJA["time"].dt.year>=1979)

fsphis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/ps_historical_r144x72_197901-201412.nc")
sphis_ver_JJA = fsphis_ver_JJA["ps"].sel(time=fsphis_ver_JJA["time"].dt.year>=1979)

divhis_ver_JJA = ca.cal_divergence(uhis_ver_JJA.sel(level=[200.0, 850.0]), vhis_ver_JJA.sel(level=[200.0, 850.0]))

## ssp585

In [4]:
fpressp585_p3 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/pr_ssp585_p3_r144x72_206401-209912.nc")
pressp585_p3_JJA = fpressp585_p3["pr"].sel(time=fpressp585_p3.time.dt.year>=2064)
pressp585_p3_JJA.attrs["units"] = "mm/day"
pressp585_p3_JJA.attrs["standard_name"] = "precipitation"

fhgtssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/zg_ssp585_p3_r144x72_206401-209912.nc")
hgtssp585_p3_ver_JJA = fhgtssp585_p3_ver_JJA["zg"].sel(time=fhgtssp585_p3_ver_JJA.time.dt.year>=2064)
# hgtssp585_p3_ver_JJA = hgtssp585_p3_ver_JJA - hgtssp585_p3_ver_JJA.mean(dim="lon", skipna=True)


fussp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ua_ssp585_p3_r144x72_206401-209912.nc")
ussp585_p3_ver_JJA = fussp585_p3_ver_JJA["ua"].sel(time=fussp585_p3_ver_JJA.time.dt.year>=2064)

fvssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/va_ssp585_p3_r144x72_206401-209912.nc")
vssp585_p3_ver_JJA = fvssp585_p3_ver_JJA["va"].sel(time=fvssp585_p3_ver_JJA.time.dt.year>=2064)

# fwssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/wap_ssp585_p3_r144x72_206401-209912.nc")
# wssp585_p3_ver_JJA = fwssp585_p3_ver_JJA["wap"].sel(time=fwssp585_p3_ver_JJA.time.dt.year>=2064)

fsstssp585_p3_MAM = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")
sstssp585_p3_MAM = fsstssp585_p3_MAM["__xarray_dataarray_variable__"].sel(time=fsstssp585_p3_MAM.time.dt.year>=2064)

fsstssp585_p3_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")
sstssp585_p3_JJA = fsstssp585_p3_JJA["__xarray_dataarray_variable__"].sel(time=fsstssp585_p3_JJA.time.dt.year>=2064)

fsstssp585_p3_SON = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")
sstssp585_p3_SON = fsstssp585_p3_SON["__xarray_dataarray_variable__"].sel(time=fsstssp585_p3_SON.time.dt.year>=2064)

fsstssp585_p3_DJF = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")
sstssp585_p3_DJF = fsstssp585_p3_DJF["__xarray_dataarray_variable__"].sel(time=fsstssp585_p3_DJF.time.dt.year>=2064)

fussp585_p3_MAM = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/u_ssp585_p3_r144x72_206401-209912.nc")
ussp585_p3_MAM = fussp585_p3_MAM["u"].sel(time=fussp585_p3_MAM.time.dt.year>=2064)

fussp585_p3_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/u_ssp585_p3_r144x72_206401-209912.nc")
ussp585_p3_JJA = fussp585_p3_JJA["u"].sel(time=fussp585_p3_JJA.time.dt.year>=2064)

fussp585_p3_SON = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/u_ssp585_p3_r144x72_206401-209912.nc")
ussp585_p3_SON = fussp585_p3_SON["u"].sel(time=fussp585_p3_SON.time.dt.year>=2064)

fussp585_p3_DJF = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/u_ssp585_p3_r144x72_206401-209912.nc")
ussp585_p3_DJF = fussp585_p3_DJF["u"].sel(time=fussp585_p3_DJF.time.dt.year>=2064)

fvssp585_p3_MAM = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/v_ssp585_p3_r144x72_206401-209912.nc")
vssp585_p3_MAM = fvssp585_p3_MAM["v"].sel(time=fvssp585_p3_MAM.time.dt.year>=2064)

fvssp585_p3_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/v_ssp585_p3_r144x72_206401-209912.nc")
vssp585_p3_JJA = fvssp585_p3_JJA["v"].sel(time=fvssp585_p3_JJA.time.dt.year>=2064)

fvssp585_p3_SON = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/v_ssp585_p3_r144x72_206401-209912.nc")
vssp585_p3_SON = fvssp585_p3_SON["v"].sel(time=fvssp585_p3_SON.time.dt.year>=2064)

fvssp585_p3_DJF = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/v_ssp585_p3_r144x72_206401-209912.nc")
vssp585_p3_DJF = fvssp585_p3_DJF["v"].sel(time=fvssp585_p3_DJF.time.dt.year>=2064)

ftssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ta_ssp585_p3_r144x72_206401-209912.nc")
tssp585_p3_ver_JJA = ftssp585_p3_ver_JJA["ta"].sel(time=ftssp585_p3_ver_JJA.time.dt.year>=2064)

spssp585_p3_MAM = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/sp_ssp585_p3_r144x72_206401-209912.nc")
spssp585_p3_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/sp_ssp585_p3_r144x72_206401-209912.nc")
spssp585_p3_SON = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/sp_ssp585_p3_r144x72_206401-209912.nc")
spssp585_p3_DJF = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/sp_ssp585_p3_r144x72_206401-209912.nc")

divssp585_p3_ver_JJA = ca.cal_divergence(ussp585_p3_ver_JJA.sel(level=[200.0, 850.0]), vssp585_p3_ver_JJA.sel(level=[200.0, 850.0]))

fuqssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ssp585_uq_dpg.nc")
uqssp585_p3_ver_JJA = fuqssp585_p3_ver_JJA["uq_dpg"].sel(time=fuqssp585_p3_ver_JJA["time"].dt.year>=2064)

fvqssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ssp585_vq_dpg.nc")
vqssp585_p3_ver_JJA = fvqssp585_p3_ver_JJA["vq_dpg"].sel(time=fvqssp585_p3_ver_JJA["time"].dt.year>=2064)

fuqvqdivssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ssp585_p3_div_uqvq_dpg.nc")
uqvqdivssp585_p3_ver_JJA = fuqvqdivssp585_p3_ver_JJA["div_uqvq"].sel(time=fuqvqdivssp585_p3_ver_JJA["time"].dt.year>=2064)

fspssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ps_ssp585_p3_r144x72_206401-209912.nc")
spssp585_p3_ver_JJA = fspssp585_p3_ver_JJA["ps"].sel(time=fspssp585_p3_ver_JJA["time"].dt.year>=2064)

# 计算Nino3.4 index并去趋势

In [5]:
# 计算Nino 3.4 index并去趋势
Nino34Had = ca.cal_area_weighted_mean(ca.filplonlat(sstHad.copy()).loc[:, -5:5, -170:-120])
Nino34Had = ca.detrend_dim(Nino34Had, "time", deg=1, demean=False)
Nino34Had_MAM, Nino34Had_JJA, Nino34Had_SON, Nino34Had_DJF = ca.mon_to_season(Nino34Had)
Nino34Had_MAM = ca.detrend_dim(Nino34Had_MAM, "time", deg=1, demean=False)
Nino34Had_JJA = ca.detrend_dim(Nino34Had_JJA, "time", deg=1, demean=False)
Nino34Had_SON = ca.detrend_dim(Nino34Had_SON, "time", deg=1, demean=False)
Nino34Had_DJF = ca.detrend_dim(Nino34Had_DJF, "time", deg=1, demean=False)
Nino34Had_MAM_nor = ca.standardize(Nino34Had_MAM)
Nino34Had_JJA_nor = ca.standardize(Nino34Had_JJA)
Nino34Had_SON_nor = ca.standardize(Nino34Had_SON)
Nino34Had_DJF_nor = ca.standardize(Nino34Had_DJF)

{'standard_name': 'sea_surface_temperature', 'long_name': 'sst', 'units': 'C', 'cell_methods': 'time: lat: lon: mean'}


Before flip, lon range is [0.0, 357.5].


After flip, lon range is [-180.0, 177.5].
{'standard_name': 'sea_surface_temperature', 'long_name': 'sst', 'units': 'C', 'cell_methods': 'time: lat: lon: mean'}


In [6]:
# 计算Nino3.4 index并去趋势
ssthis_nondetrend = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/original/non_detrend/tos_historical_r144x72_195001-201412.nc")
sstssp585_nondetrend = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/original/non_detrend/tos_ssp585_r144x72_201501-209912.nc")

Nino34CMIP6_MAM, Nino34CMIP6_JJA, Nino34CMIP6_SON, Nino34CMIP6_DJF = ca.mon_to_season(ca.cal_area_weighted_mean(ca.filplonlat(xr.concat([ssthis_nondetrend.sel(time=ssthis_nondetrend.time.dt.year>=1979), sstssp585_nondetrend], "time")).loc[:,:,-5:5,-170:-120]))

Nino34CMIP6_MAM_detrend = ca.detrend_dim(Nino34CMIP6_MAM, "time", deg=1, demean=True)
Nino34CMIP6_JJA_detrend = ca.detrend_dim(Nino34CMIP6_JJA, "time", deg=1, demean=True)
Nino34CMIP6_SON_detrend = ca.detrend_dim(Nino34CMIP6_SON, "time", deg=1, demean=True)
Nino34CMIP6_DJF_detrend = ca.detrend_dim(Nino34CMIP6_DJF, "time", deg=1, demean=True)
Nino34his_MAM = ca.detrend_dim(Nino34CMIP6_MAM_detrend.sel(time=Nino34CMIP6_MAM_detrend.time.dt.year<=2014), "time", deg=1, demean=True)
Nino34his_JJA = ca.detrend_dim(Nino34CMIP6_JJA_detrend.sel(time=Nino34CMIP6_JJA_detrend.time.dt.year<=2014), "time", deg=1, demean=True)
Nino34his_SON = ca.detrend_dim(Nino34CMIP6_SON_detrend.sel(time=Nino34CMIP6_SON_detrend.time.dt.year<=2014), "time", deg=1, demean=True)
Nino34his_DJF = ca.detrend_dim(Nino34CMIP6_DJF_detrend.sel(time=Nino34CMIP6_DJF_detrend.time.dt.year<=2014), "time", deg=1, demean=True)

Nino34ssp585_p3_MAM = ca.detrend_dim(Nino34CMIP6_MAM_detrend.sel(time=Nino34CMIP6_MAM_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
Nino34ssp585_p3_JJA = ca.detrend_dim(Nino34CMIP6_JJA_detrend.sel(time=Nino34CMIP6_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
Nino34ssp585_p3_SON = ca.detrend_dim(Nino34CMIP6_SON_detrend.sel(time=Nino34CMIP6_SON_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
Nino34ssp585_p3_DJF = ca.detrend_dim(Nino34CMIP6_DJF_detrend.sel(time=Nino34CMIP6_DJF_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

Nino34his_MAM_nor = ca.standardize(Nino34his_MAM.copy())
Nino34his_JJA_nor = ca.standardize(Nino34his_JJA.copy())
Nino34his_SON_nor = ca.standardize(Nino34his_SON.copy())
Nino34his_DJF_nor = ca.standardize(Nino34his_DJF.copy())

Nino34ssp585_p3_MAM_nor = ca.standardize(Nino34ssp585_p3_MAM.copy())
Nino34ssp585_p3_JJA_nor = ca.standardize(Nino34ssp585_p3_JJA.copy())
Nino34ssp585_p3_SON_nor = ca.standardize(Nino34ssp585_p3_SON.copy())
Nino34ssp585_p3_DJF_nor = ca.standardize(Nino34ssp585_p3_DJF.copy())

{'standard_name': 'sea_surface_temperature', 'long_name': 'Sea Surface Temperature', 'units': 'degC', 'comment': 'Temperature of upper boundary of the liquid ocean, including temperatures below sea-ice and floating ice shelves.', 'cell_methods': 'area: mean where sea time: mean', 'cell_measures': 'area: areacello', 'history': '2019-11-08T18:45:39Z altered by CMOR: replaced missing value flag (-1e+20) with standard missing value (1e+20).'}


Before flip, lon range is [0.0, 357.5].


After flip, lon range is [-180.0, 177.5].
{'standard_name': 'sea_surface_temperature', 'long_name': 'Sea Surface Temperature', 'units': 'degC', 'comment': 'Temperature of upper boundary of the liquid ocean, including temperatures below sea-ice and floating ice shelves.', 'cell_methods': 'area: mean where sea time: mean', 'cell_measures': 'area: areacello', 'history': '2019-11-08T18:45:39Z altered by CMOR: replaced missing value flag (-1e+20) with standard missing value (1e+20).'}


# 计算印度分区降水

In [7]:
lat = prehis_JJA.coords["lat"]
lon = prehis_JJA.coords["lon"]
# 北部
nIndia_N = 35.0
nIndia_S = 20.0
nIndia_W = 70.0
nIndia_E = 85.0
lat_nIndia_range = lat[(lat >= nIndia_S) & (lat <= nIndia_N)]
lon_nIndia_range = lon[(lon >= nIndia_W) & (lon <= nIndia_E)]

preGPCP_nIndia_JJA = ca.cal_area_weighted_mean(preGPCP_JJA_detrend.sel(lat=lat_nIndia_range, lon=lon_nIndia_range))
prehis_nIndia_JJA = ca.cal_area_weighted_mean(prehis_JJA.sel(lat=lat_nIndia_range, lon=lon_nIndia_range))
pressp585_p3_nIndia_JJA = ca.cal_area_weighted_mean(pressp585_p3_JJA.sel(lat=lat_nIndia_range, lon=lon_nIndia_range))

# detrend
preGPCP_nIndia_JJA_detrend = ca.detrend_dim(preGPCP_nIndia_JJA.copy(), "time", deg=1, demean=True)
prehis_nIndia_JJA_detrend = ca.detrend_dim(prehis_nIndia_JJA.copy(), "time", deg=1, demean=True)
pressp585_p3_nIndia_JJA_detrend = ca.detrend_dim(pressp585_p3_nIndia_JJA.copy(), "time", deg=1, demean=True)

# standardize
preGPCP_nIndia_JJA_nor = ca.standardize(preGPCP_nIndia_JJA_detrend)
prehis_nIndia_JJA_nor = ca.standardize(prehis_nIndia_JJA_detrend)
pressp585_p3_nIndia_JJA_nor = ca.standardize(pressp585_p3_nIndia_JJA_detrend)

# 南部
sIndia_N = 20.0
sIndia_S = 5.0
sIndia_W = 70.0
sIndia_E = 85.0
lat_sIndia_range = lat[(lat >= sIndia_S) & (lat <= sIndia_N)]
lon_sIndia_range = lon[(lon >= sIndia_W) & (lon <= sIndia_E)]

preGPCP_sIndia_JJA = ca.cal_area_weighted_mean(preGPCP_JJA_detrend.sel(lat=lat_sIndia_range, lon=lon_sIndia_range))
prehis_sIndia_JJA = ca.cal_area_weighted_mean(prehis_JJA.sel(lat=lat_sIndia_range, lon=lon_sIndia_range))
pressp585_p3_sIndia_JJA = ca.cal_area_weighted_mean(pressp585_p3_JJA.sel(lat=lat_sIndia_range, lon=lon_sIndia_range))

# detrend
preGPCP_sIndia_JJA_detrend = ca.detrend_dim(preGPCP_sIndia_JJA.copy(), "time", deg=1, demean=True)
prehis_sIndia_JJA_detrend = ca.detrend_dim(prehis_sIndia_JJA.copy(), "time", deg=1, demean=True)
pressp585_p3_sIndia_JJA_detrend = ca.detrend_dim(pressp585_p3_sIndia_JJA.copy(), "time", deg=1, demean=True)

# standardize
preGPCP_sIndia_JJA_nor = ca.standardize(preGPCP_sIndia_JJA_detrend)
prehis_sIndia_JJA_nor = ca.standardize(prehis_sIndia_JJA_detrend)
pressp585_p3_sIndia_JJA_nor = ca.standardize(pressp585_p3_sIndia_JJA_detrend)

# 分离与前冬Nino3.4 index相关或无关的印度南部降水

In [8]:
tmp = preGPCP_sIndia_JJA_nor[1:].copy()
tmp.coords["time"] = Nino34Had_DJF_nor.time
linre_DJFnino34_sIndR = stats.linregress(Nino34Had_DJF_nor, tmp)
sIndR_GPCP_ENSO_related = linre_DJFnino34_sIndR[0]*Nino34Had_DJF_nor+linre_DJFnino34_sIndR[1]
sIndR_GPCP_ENSO_unrelated = tmp - sIndR_GPCP_ENSO_related
del(tmp)
sIndR_GPCP_ENSO_related_nor = ca.standardize(sIndR_GPCP_ENSO_related)
sIndR_GPCP_ENSO_unrelated_nor = ca.standardize(sIndR_GPCP_ENSO_unrelated)
sIndR_GPCP_ENSO_related_nor.coords["time"] = preGPCP_sIndia_JJA_nor[1:].time
sIndR_GPCP_ENSO_unrelated_nor.coords["time"] = preGPCP_sIndia_JJA_nor[1:].time

In [9]:
tmp = prehis_sIndia_JJA_nor[:,1:].copy()
tmp.coords["time"] = Nino34his_DJF_nor.time
linre_his_DJFnino34_sIndR = ca.dim_linregress(Nino34his_DJF_nor, tmp)
sIndR_his_ENSO_related = linre_his_DJFnino34_sIndR[:][0]*Nino34his_DJF_nor+linre_his_DJFnino34_sIndR[:][1]
sIndR_his_ENSO_unrelated = tmp - sIndR_his_ENSO_related
del(tmp)
sIndR_his_ENSO_related_nor = ca.standardize(sIndR_his_ENSO_related)
sIndR_his_ENSO_unrelated_nor = ca.standardize(sIndR_his_ENSO_unrelated)

sIndR_his_ENSO_related_nor.coords["time"] = prehis_sIndia_JJA_nor[:,1:].time
sIndR_his_ENSO_unrelated_nor.coords["time"] = prehis_sIndia_JJA_nor[:,1:].time

In [15]:
tmp = pressp585_p3_sIndia_JJA_nor.copy()
tmp.coords["time"] = Nino34ssp585_p3_DJF_nor.time
linre_ssp585_p3_DJFnino34_sIndR = ca.dim_linregress(Nino34ssp585_p3_DJF_nor, tmp)
sIndR_ssp585_p3_ENSO_related = linre_ssp585_p3_DJFnino34_sIndR[:][0]*Nino34ssp585_p3_DJF_nor+linre_ssp585_p3_DJFnino34_sIndR[:][1]
sIndR_ssp585_p3_ENSO_unrelated = tmp - sIndR_ssp585_p3_ENSO_related
del(tmp)
sIndR_ssp585_p3_ENSO_related_nor = ca.standardize(sIndR_ssp585_p3_ENSO_related)
sIndR_ssp585_p3_ENSO_unrelated_nor = ca.standardize(sIndR_ssp585_p3_ENSO_unrelated)

sIndR_ssp585_p3_ENSO_related_nor.coords["time"] = pressp585_p3_sIndia_JJA_nor.time
sIndR_ssp585_p3_ENSO_unrelated_nor.coords["time"] = pressp585_p3_sIndia_JJA_nor.time

# 计算与ENSO相关的环流

## 观测

In [ ]:
# 观测
pre_north_south_ENSO_related_unrelated_avalue, pre_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(preGPCP_JJA[1:], [preGPCP_nIndia_JJA_nor[1:], sIndR_GPCP_ENSO_related_nor, sIndR_GPCP_ENSO_unrelated_nor], concat_dim="area")
hgt_north_south_ENSO_related_unrelated_avalue, hgt_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(hgtERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0])[1:], [preGPCP_nIndia_JJA_nor[1:], sIndR_GPCP_ENSO_related_nor, sIndR_GPCP_ENSO_unrelated_nor], concat_dim="area")

In [ ]:
tmp = sstHad_DJF.copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
sstDJF_north_south_ENSO_related_unrelated_slope, sstDJF_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], sIndR_GPCP_ENSO_related_nor, sIndR_GPCP_ENSO_unrelated_nor], concat_dim="area")

tmp = sstHad_MAM[1:].copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
sstMAM_north_south_ENSO_related_unrelated_slope, sstMAM_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], sIndR_GPCP_ENSO_related_nor, sIndR_GPCP_ENSO_unrelated_nor], concat_dim="area")

tmp = sstHad_JJA[1:].copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
sstJJA_north_south_ENSO_related_unrelated_slope, sstJJA_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], sIndR_GPCP_ENSO_related_nor, sIndR_GPCP_ENSO_unrelated_nor], concat_dim="area")

tmp = sstHad_SON[1:].copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
sstSON_north_south_ENSO_related_unrelated_slope, sstSON_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], sIndR_GPCP_ENSO_related_nor, sIndR_GPCP_ENSO_unrelated_nor], concat_dim="area")

tmp = sstHad_DJF[1:].copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:-1].time
sstnDJF_north_south_ENSO_related_unrelated_slope, sstnDJF_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:-1], sIndR_GPCP_ENSO_related_nor[:-1], sIndR_GPCP_ENSO_unrelated_nor[:-1]], concat_dim="area")


In [18]:
tmp = uERA5_ver_DJF.sel(level=[200.0, 500.0, 850.0, 1000.0]).copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
uDJF_north_south_ENSO_related_unrelated_slope, uDJF_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], sIndR_GPCP_ENSO_related_nor, sIndR_GPCP_ENSO_unrelated_nor], concat_dim="area")

tmp = uERA5_ver_MAM.sel(level=[200.0, 500.0, 850.0, 1000.0])[1:].copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
uMAM_north_south_ENSO_related_unrelated_slope, uMAM_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], sIndR_GPCP_ENSO_related_nor, sIndR_GPCP_ENSO_unrelated_nor], concat_dim="area")

tmp = uERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0, 1000.0])[1:].copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
uJJA_north_south_ENSO_related_unrelated_slope, uJJA_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], sIndR_GPCP_ENSO_related_nor, sIndR_GPCP_ENSO_unrelated_nor], concat_dim="area")

tmp = uERA5_ver_SON.sel(level=[200.0, 500.0, 850.0, 1000.0])[1:].copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
uSON_north_south_ENSO_related_unrelated_slope, uSON_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], sIndR_GPCP_ENSO_related_nor, sIndR_GPCP_ENSO_unrelated_nor], concat_dim="area")

tmp = uERA5_ver_DJF.sel(level=[200.0, 500.0, 850.0, 1000.0])[1:].copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:-1].time
unDJF_north_south_ENSO_related_unrelated_slope, unDJF_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:-1], sIndR_GPCP_ENSO_related_nor[:-1], sIndR_GPCP_ENSO_unrelated_nor[:-1]], concat_dim="area")

tmp = vERA5_ver_DJF.sel(level=[200.0, 500.0, 850.0, 1000.0]).copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
vDJF_north_south_ENSO_related_unrelated_slope, vDJF_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], sIndR_GPCP_ENSO_related_nor, sIndR_GPCP_ENSO_unrelated_nor], concat_dim="area")

tmp = vERA5_ver_MAM.sel(level=[200.0, 500.0, 850.0, 1000.0])[1:].copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
vMAM_north_south_ENSO_related_unrelated_slope, vMAM_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], sIndR_GPCP_ENSO_related_nor, sIndR_GPCP_ENSO_unrelated_nor], concat_dim="area")

tmp = vERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0, 1000.0])[1:].copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
vJJA_north_south_ENSO_related_unrelated_slope, vJJA_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], sIndR_GPCP_ENSO_related_nor, sIndR_GPCP_ENSO_unrelated_nor], concat_dim="area")

tmp = vERA5_ver_SON.sel(level=[200.0, 500.0, 850.0, 1000.0])[1:].copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
vSON_north_south_ENSO_related_unrelated_slope, vSON_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], sIndR_GPCP_ENSO_related_nor, sIndR_GPCP_ENSO_unrelated_nor], concat_dim="area")

tmp = vERA5_ver_DJF.sel(level=[200.0, 500.0, 850.0, 1000.0])[1:].copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:-1].time
vnDJF_north_south_ENSO_related_unrelated_slope, vnDJF_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:-1], sIndR_GPCP_ENSO_related_nor[:-1], sIndR_GPCP_ENSO_unrelated_nor[:-1]], concat_dim="area")

In [ ]:
tmp = spERA5_DJF.copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
spDJF_north_south_ENSO_related_unrelated_slope, spDJF_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], sIndR_GPCP_ENSO_related_nor, sIndR_GPCP_ENSO_unrelated_nor], concat_dim="area")

tmp = spERA5_MAM[1:].copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
spMAM_north_south_ENSO_related_unrelated_slope, spMAM_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], sIndR_GPCP_ENSO_related_nor, sIndR_GPCP_ENSO_unrelated_nor], concat_dim="area")

tmp = spERA5_JJA[1:].copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
spJJA_north_south_ENSO_related_unrelated_slope, spJJA_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], sIndR_GPCP_ENSO_related_nor, sIndR_GPCP_ENSO_unrelated_nor], concat_dim="area")

tmp = spERA5_SON[1:].copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
spSON_north_south_ENSO_related_unrelated_slope, spSON_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], sIndR_GPCP_ENSO_related_nor, sIndR_GPCP_ENSO_unrelated_nor], concat_dim="area")

tmp = spERA5_DJF[1:].copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:-1].time
spnDJF_north_south_ENSO_related_unrelated_slope, spnDJF_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:-1], sIndR_GPCP_ENSO_related_nor[:-1], sIndR_GPCP_ENSO_unrelated_nor[:-1]], concat_dim="area")

In [ ]:
pre_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], pre_north_south_ENSO_related_unrelated_avalue.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], pre_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="precipitation partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)

hgt_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], hgt_north_south_ENSO_related_unrelated_avalue.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], hgt_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        level=hgt_north_south_ENSO_related_unrelated_avalue.level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/pre_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/pre_north_south_ENSO_related_unrelated_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/hgt_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/hgt_north_south_ENSO_related_unrelated_partial_correlation.nc")

pre_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/pre_north_south_ENSO_related_unrelated_partial_correlation.nc")
hgt_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/hgt_north_south_ENSO_related_unrelated_partial_correlation.nc")

sstDJF_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], sstDJF_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], sstDJF_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="DJF SST partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)

sstMAM_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], sstMAM_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], sstMAM_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="MAM SST partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)

sstJJA_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], sstJJA_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], sstJJA_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="JJA SST partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)
sstSON_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], sstSON_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], sstSON_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="SON SST partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)
sstnDJF_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], sstnDJF_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], sstnDJF_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="nDJF SST partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstDJF_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstDJF_north_south_ENSO_related_unrelated_partial_correlation.nc")
sstDJF_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstDJF_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstMAM_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstMAM_north_south_ENSO_related_unrelated_partial_correlation.nc")
sstMAM_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstMAM_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstJJA_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstJJA_north_south_ENSO_related_unrelated_partial_correlation.nc")
sstJJA_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstJJA_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstSON_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstSON_north_south_ENSO_related_unrelated_partial_correlation.nc")
sstSON_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstSON_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstnDJF_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstnDJF_north_south_ENSO_related_unrelated_partial_correlation.nc")
sstnDJF_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstnDJF_north_south_ENSO_related_unrelated_partial_correlation.nc")

In [20]:
uDJF_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], uDJF_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], uDJF_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        level=np.array([200.0, 500.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="DJF u partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)

uMAM_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], uMAM_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], uMAM_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        level=np.array([200.0, 500.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="MAM u partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)

uJJA_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], uJJA_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], uJJA_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        level=np.array([200.0, 500.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="JJA u partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)
uSON_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], uSON_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], uSON_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        level=np.array([200.0, 500.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="SON u partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)
unDJF_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], unDJF_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], unDJF_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        level=np.array([200.0, 500.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="nDJF u partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/uDJF_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/uDJF_north_south_ENSO_related_unrelated_partial_correlation.nc")
uDJF_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/uDJF_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/uMAM_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/uMAM_north_south_ENSO_related_unrelated_partial_correlation.nc")
uMAM_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/uMAM_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/uJJA_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/uJJA_north_south_ENSO_related_unrelated_partial_correlation.nc")
uJJA_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/uJJA_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/uSON_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/uSON_north_south_ENSO_related_unrelated_partial_correlation.nc")
uSON_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/uSON_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/unDJF_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/unDJF_north_south_ENSO_related_unrelated_partial_correlation.nc")
unDJF_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/unDJF_north_south_ENSO_related_unrelated_partial_correlation.nc")

vDJF_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], vDJF_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], vDJF_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        level=np.array([200.0, 500.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="DJF v partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)

vMAM_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], vMAM_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], vMAM_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        level=np.array([200.0, 500.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="MAM v partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)

vJJA_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], vJJA_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], vJJA_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        level=np.array([200.0, 500.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="JJA v partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)
vSON_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], vSON_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], vSON_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        level=np.array([200.0, 500.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="SON v partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)
vnDJF_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], vnDJF_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], vnDJF_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        level=np.array([200.0, 500.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="nDJF v partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vDJF_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vDJF_north_south_ENSO_related_unrelated_partial_correlation.nc")
vDJF_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vDJF_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vMAM_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vMAM_north_south_ENSO_related_unrelated_partial_correlation.nc")
vMAM_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vMAM_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vJJA_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vJJA_north_south_ENSO_related_unrelated_partial_correlation.nc")
vJJA_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vJJA_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vSON_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vSON_north_south_ENSO_related_unrelated_partial_correlation.nc")
vSON_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vSON_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vnDJF_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vnDJF_north_south_ENSO_related_unrelated_partial_correlation.nc")
vnDJF_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vnDJF_north_south_ENSO_related_unrelated_partial_correlation.nc")

spDJF_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], spDJF_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], spDJF_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="DJF sp partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)

spMAM_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], spMAM_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], spMAM_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="MAM sp partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)

spJJA_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], spJJA_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], spJJA_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="JJA sp partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)
spSON_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], spSON_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], spSON_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="SON sp partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)
spnDJF_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], spnDJF_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], spnDJF_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="nDJF sp partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spDJF_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spDJF_north_south_ENSO_related_unrelated_partial_correlation.nc")
spDJF_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spDJF_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spMAM_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spMAM_north_south_ENSO_related_unrelated_partial_correlation.nc")
spMAM_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spMAM_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spJJA_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spJJA_north_south_ENSO_related_unrelated_partial_correlation.nc")
spJJA_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spJJA_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spSON_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spSON_north_south_ENSO_related_unrelated_partial_correlation.nc")
spSON_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spSON_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spnDJF_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spnDJF_north_south_ENSO_related_unrelated_partial_correlation.nc")
spnDJF_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spnDJF_north_south_ENSO_related_unrelated_partial_correlation.nc")

## 模式

### pre hgt uq vq uqvqdiv historical 

In [ ]:
# historical
pre_his_north_south_ENSO_related_unrelated_slope, pre_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(prehis_JJA[:,1:], [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")
hgt_his_north_south_ENSO_related_unrelated_avalue, hgt_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(hgthis_ver_JJA.sel(level=[200.0, 500.0, 850.0])[:,1:], [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")
uq_his_north_south_ENSO_related_unrelated_avalue, uq_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(uqhis_ver_JJA[:,1:], [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")
vq_his_north_south_ENSO_related_unrelated_avalue, vq_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(vqhis_ver_JJA[:,1:], [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")
uqvqdiv_his_north_south_ENSO_related_unrelated_avalue, uqvqdiv_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(uqvqdivhis_ver_JJA[:,1:], [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")


### pre hgt uq vq uqvqdiv ssp585

In [18]:
# ssp585
pre_ssp585_p3_north_south_ENSO_related_unrelated_slope, pre_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(pressp585_p3_JJA, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")
hgt_ssp585_p3_north_south_ENSO_related_unrelated_avalue, hgt_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(hgtssp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]), [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")
uq_ssp585_p3_north_south_ENSO_related_unrelated_avalue, uq_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(uqssp585_p3_ver_JJA, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")
vq_ssp585_p3_north_south_ENSO_related_unrelated_avalue, vq_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(vqssp585_p3_ver_JJA, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")
uqvqdiv_ssp585_p3_north_south_ENSO_related_unrelated_avalue, uqvqdiv_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(uqvqdivssp585_p3_ver_JJA, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")

In [ ]:
# historical
pre_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], pre_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], pre_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=pre_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="precipitation partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)
hgt_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], hgt_his_north_south_ENSO_related_unrelated_avalue.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], hgt_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=hgt_his_north_south_ENSO_related_unrelated_avalue.coords["models"].data,
        level=hgt_his_north_south_ENSO_related_unrelated_avalue.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt partially regression onto north or south ENSO_related or south ENSO_unrelated hgt"),
)


if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/pre_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/pre_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/hgt_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/hgt_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

pre_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/pre_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
hgt_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/hgt_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

uq_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], uq_his_north_south_ENSO_related_unrelated_avalue.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], uq_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=uq_his_north_south_ENSO_related_unrelated_avalue.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uq partially regression onto north or south ENSO_related or south ENSO_unrelated uq"),
)
vq_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], vq_his_north_south_ENSO_related_unrelated_avalue.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], vq_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=vq_his_north_south_ENSO_related_unrelated_avalue.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vq partially regression onto north or south ENSO_related or south ENSO_unrelated vq"),
)
uqvqdiv_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], uqvqdiv_his_north_south_ENSO_related_unrelated_avalue.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], uqvqdiv_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=uqvqdiv_his_north_south_ENSO_related_unrelated_avalue.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uqvqdiv partially regression onto north or south ENSO_related or south ENSO_unrelated uqvqdiv"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uq_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uq_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vq_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vq_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uqvqdiv_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uqvqdiv_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

uq_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uq_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
vq_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vq_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
uqvqdiv_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uqvqdiv_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

In [19]:

# ssp585
pre_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], pre_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], pre_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=pre_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="precipitation partially regression onto north or south ENSO_related or south ENSO_unrelated precipitation"),
)
hgt_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], hgt_ssp585_p3_north_south_ENSO_related_unrelated_avalue.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], hgt_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=hgt_ssp585_p3_north_south_ENSO_related_unrelated_avalue.coords["models"].data,
        level=hgt_ssp585_p3_north_south_ENSO_related_unrelated_avalue.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt partially regression onto north or south ENSO_related or south ENSO_unrelated hgt"),
)


if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/pre_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/pre_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/hgt_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/hgt_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

pre_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/pre_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
hgt_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/hgt_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")



uq_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], uq_ssp585_p3_north_south_ENSO_related_unrelated_avalue.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], uq_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=uq_ssp585_p3_north_south_ENSO_related_unrelated_avalue.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uq partially regression onto north or south ENSO_related or south ENSO_unrelated uq"),
)
vq_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], vq_ssp585_p3_north_south_ENSO_related_unrelated_avalue.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], vq_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=vq_ssp585_p3_north_south_ENSO_related_unrelated_avalue.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vq partially regression onto north or south ENSO_related or south ENSO_unrelated vq"),
)
uqvqdiv_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], uqvqdiv_ssp585_p3_north_south_ENSO_related_unrelated_avalue.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], uqvqdiv_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=uqvqdiv_ssp585_p3_north_south_ENSO_related_unrelated_avalue.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uqvqdiv partially regression onto north or south ENSO_related or south ENSO_unrelated uqvqdiv"),
)


if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uq_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uq_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vq_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vq_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uqvqdiv_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uqvqdiv_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

uq_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uq_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
vq_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vq_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
uqvqdiv_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uqvqdiv_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

### sst historical

In [ ]:
# historical
tmp = ssthis_DJF.copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
sstDJF_his_north_south_ENSO_related_unrelated_slope, sstDJF_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")

tmp = ssthis_MAM[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
sstMAM_his_north_south_ENSO_related_unrelated_slope, sstMAM_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")

tmp = ssthis_JJA[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
sstJJA_his_north_south_ENSO_related_unrelated_slope, sstJJA_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")

tmp = ssthis_SON[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
sstSON_his_north_south_ENSO_related_unrelated_slope, sstSON_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")

tmp = ssthis_DJF[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:-1].time
sstnDJF_his_north_south_ENSO_related_unrelated_slope, sstnDJF_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:-1], sIndR_his_ENSO_related_nor[:,:-1], sIndR_his_ENSO_unrelated_nor[:,:-1]], concat_dim="area")

### sst ssp585

In [21]:
# ssp585
tmp = sstssp585_p3_DJF.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
sstDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope, sstDJF_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")

tmp = sstssp585_p3_MAM.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
sstMAM_ssp585_p3_north_south_ENSO_related_unrelated_slope, sstMAM_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")

tmp = sstssp585_p3_JJA.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
sstJJA_ssp585_p3_north_south_ENSO_related_unrelated_slope, sstJJA_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")

tmp = sstssp585_p3_SON.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
sstSON_ssp585_p3_north_south_ENSO_related_unrelated_slope, sstSON_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")

tmp = sstssp585_p3_DJF[:,1:].copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor[:,:-1].time
sstnDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope, sstnDJF_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor[:,:-1], sIndR_ssp585_p3_ENSO_related_nor[:,:-1], sIndR_ssp585_p3_ENSO_unrelated_nor[:,:-1]], concat_dim="area")

In [ ]:
# historical
sstDJF_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstDJF_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstDJF_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=sstDJF_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstDJF partially regression onto north or south ENSO_related or south ENSO_unrelated sstDJF"),
)

sstMAM_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstMAM_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstMAM_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=sstMAM_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstMAM partially regression onto north or south ENSO_related or south ENSO_unrelated sstMAM"),
)

sstJJA_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstJJA_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstJJA_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=sstJJA_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstJJA partially regression onto north or south ENSO_related or south ENSO_unrelated sstJJA"),
)

sstSON_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstSON_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstSON_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=sstSON_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstSON partially regression onto north or south ENSO_related or south ENSO_unrelated sstSON"),
)

sstnDJF_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstnDJF_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstnDJF_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=sstnDJF_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstnDJF partially regression onto north or south ENSO_related or south ENSO_unrelated sstnDJF"),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
sstDJF_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstMAM_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstMAM_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
sstMAM_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstMAM_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstJJA_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstJJA_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
sstJJA_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstJJA_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstSON_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstSON_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
sstSON_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstSON_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstnDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstnDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
sstnDJF_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstnDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc")


In [23]:

# ssp585
sstDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstDJF_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=sstDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstDJF partially regression onto north or south ENSO_related or south ENSO_unrelated sstDJF"),
)

sstMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstMAM_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstMAM_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=sstMAM_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstMAM partially regression onto north or south ENSO_related or south ENSO_unrelated sstMAM"),
)

sstJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstJJA_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstJJA_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=sstJJA_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstJJA partially regression onto north or south ENSO_related or south ENSO_unrelated sstJJA"),
)

sstSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstSON_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstSON_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=sstSON_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstSON partially regression onto north or south ENSO_related or south ENSO_unrelated sstSON"),
)

sstnDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstnDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstnDJF_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=sstnDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstnDJF partially regression onto north or south ENSO_related or south ENSO_unrelated sstnDJF"),
)
# ssp585
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
sstDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
sstMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
sstJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
sstSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstnDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstnDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
sstnDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstnDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

### uv historical

In [ ]:
# historical
tmp = uhis_DJF.copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
uDJF_his_north_south_ENSO_related_unrelated_slope, uDJF_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")

tmp = uhis_MAM[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
uMAM_his_north_south_ENSO_related_unrelated_slope, uMAM_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")

tmp = uhis_JJA[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
uJJA_his_north_south_ENSO_related_unrelated_slope, uJJA_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")

tmp = uhis_SON[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
uSON_his_north_south_ENSO_related_unrelated_slope, uSON_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")

tmp = uhis_DJF[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:-1].time
unDJF_his_north_south_ENSO_related_unrelated_slope, unDJF_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:-1], sIndR_his_ENSO_related_nor[:,:-1], sIndR_his_ENSO_unrelated_nor[:,:-1]], concat_dim="area")

# historical
tmp = vhis_DJF.copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
vDJF_his_north_south_ENSO_related_unrelated_slope, vDJF_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")

tmp = vhis_MAM[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
vMAM_his_north_south_ENSO_related_unrelated_slope, vMAM_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")

tmp = vhis_JJA[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
vJJA_his_north_south_ENSO_related_unrelated_slope, vJJA_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")

tmp = vhis_SON[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
vSON_his_north_south_ENSO_related_unrelated_slope, vSON_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")

tmp = vhis_DJF[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:-1].time
vnDJF_his_north_south_ENSO_related_unrelated_slope, vnDJF_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:-1], sIndR_his_ENSO_related_nor[:,:-1], sIndR_his_ENSO_unrelated_nor[:,:-1]], concat_dim="area")

In [ ]:
uDJF_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], uDJF_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], uDJF_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=uDJF_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uDJF partially regression onto north or south ENSO_related or south ENSO_unrelated uDJF"),
)

uMAM_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], uMAM_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], uMAM_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=uMAM_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uMAM partially regression onto north or south ENSO_related or south ENSO_unrelated uMAM"),
)

uJJA_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], uJJA_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], uJJA_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=uJJA_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uJJA partially regression onto north or south ENSO_related or south ENSO_unrelated uJJA"),
)

uSON_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], uSON_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], uSON_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=uSON_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uSON partially regression onto north or south ENSO_related or south ENSO_unrelated uSON"),
)

unDJF_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], unDJF_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], unDJF_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=unDJF_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="unDJF partially regression onto north or south ENSO_related or south ENSO_unrelated unDJF"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
uDJF_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uMAM_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uMAM_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
uMAM_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uMAM_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uJJA_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uJJA_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
uJJA_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uJJA_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uSON_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uSON_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
uSON_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uSON_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/unDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/unDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
unDJF_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/unDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

vDJF_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], vDJF_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], vDJF_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=vDJF_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vDJF partially regression onto north or south ENSO_related or south ENSO_unrelated vDJF"),
)

vMAM_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], vMAM_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], vMAM_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=vMAM_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vMAM partially regression onto north or south ENSO_related or south ENSO_unrelated vMAM"),
)

vJJA_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], vJJA_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], vJJA_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=vJJA_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vJJA partially regression onto north or south ENSO_related or south ENSO_unrelated vJJA"),
)

vSON_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], vSON_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], vSON_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=vSON_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vSON partially regression onto north or south ENSO_related or south ENSO_unrelated vSON"),
)

vnDJF_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], vnDJF_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], vnDJF_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=vnDJF_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vnDJF partially regression onto north or south ENSO_related or south ENSO_unrelated vnDJF"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
vDJF_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vMAM_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vMAM_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
vMAM_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vMAM_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vJJA_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vJJA_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
vJJA_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vJJA_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vSON_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vSON_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
vSON_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vSON_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vnDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vnDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
vnDJF_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vnDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

### uv ssp585

In [24]:
# ssp585_p3
tmp = ussp585_p3_DJF.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
uDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope, uDJF_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")

tmp = ussp585_p3_MAM.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
uMAM_ssp585_p3_north_south_ENSO_related_unrelated_slope, uMAM_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")

tmp = ussp585_p3_JJA.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
uJJA_ssp585_p3_north_south_ENSO_related_unrelated_slope, uJJA_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")

tmp = ussp585_p3_SON.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
uSON_ssp585_p3_north_south_ENSO_related_unrelated_slope, uSON_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")

tmp = ussp585_p3_DJF[:,1:].copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor[:,:-1].time
unDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope, unDJF_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor[:,:-1], sIndR_ssp585_p3_ENSO_related_nor[:,:-1], sIndR_ssp585_p3_ENSO_unrelated_nor[:,:-1]], concat_dim="area")

# ssp585_p3
tmp = vssp585_p3_DJF.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
vDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope, vDJF_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")

tmp = vssp585_p3_MAM.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
vMAM_ssp585_p3_north_south_ENSO_related_unrelated_slope, vMAM_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")

tmp = vssp585_p3_JJA.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
vJJA_ssp585_p3_north_south_ENSO_related_unrelated_slope, vJJA_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")

tmp = vssp585_p3_SON.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
vSON_ssp585_p3_north_south_ENSO_related_unrelated_slope, vSON_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")

tmp = vssp585_p3_DJF[:,1:].copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor[:,:-1].time
vnDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope, vnDJF_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor[:,:-1], sIndR_ssp585_p3_ENSO_related_nor[:,:-1], sIndR_ssp585_p3_ENSO_unrelated_nor[:,:-1]], concat_dim="area")

In [ ]:
uDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], uDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], uDJF_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=uDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uDJF partially regression onto north or south ENSO_related or south ENSO_unrelated uDJF"),
)

uMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], uMAM_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], uMAM_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=uMAM_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uMAM partially regression onto north or south ENSO_related or south ENSO_unrelated uMAM"),
)

uJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], uJJA_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], uJJA_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=uJJA_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uJJA partially regression onto north or south ENSO_related or south ENSO_unrelated uJJA"),
)

uSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], uSON_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], uSON_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=uSON_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uSON partially regression onto north or south ENSO_related or south ENSO_unrelated uSON"),
)

unDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], unDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], unDJF_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=unDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="unDJF partially regression onto north or south ENSO_related or south ENSO_unrelated unDJF"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
uDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
uMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
uJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
uSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/uSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/unDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/unDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
unDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/unDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

vDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], vDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], vDJF_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=vDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vDJF partially regression onto north or south ENSO_related or south ENSO_unrelated vDJF"),
)

vMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], vMAM_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], vMAM_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=vMAM_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vMAM partially regression onto north or south ENSO_related or south ENSO_unrelated vMAM"),
)

vJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], vJJA_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], vJJA_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=vJJA_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vJJA partially regression onto north or south ENSO_related or south ENSO_unrelated vJJA"),
)

vSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], vSON_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], vSON_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=vSON_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vSON partially regression onto north or south ENSO_related or south ENSO_unrelated vSON"),
)

vnDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], vnDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], vnDJF_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=vnDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        level=np.array([1000.0, 925.0, 850.0, 500.0, 200.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vnDJF partially regression onto north or south ENSO_related or south ENSO_unrelated vnDJF"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
vDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
vMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
vJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
vSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vnDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vnDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
vnDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/vnDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

### sp historical

In [14]:
# historical
tmp = sphis_DJF.copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
spDJF_his_north_south_ENSO_related_unrelated_slope, spDJF_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")

tmp = sphis_MAM[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
spMAM_his_north_south_ENSO_related_unrelated_slope, spMAM_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")

tmp = sphis_JJA[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
spJJA_his_north_south_ENSO_related_unrelated_slope, spJJA_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")

tmp = sphis_SON[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
spSON_his_north_south_ENSO_related_unrelated_slope, spSON_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], sIndR_his_ENSO_related_nor, sIndR_his_ENSO_unrelated_nor], concat_dim="area")

tmp = sphis_DJF[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:-1].time
spnDJF_his_north_south_ENSO_related_unrelated_slope, spnDJF_his_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:-1], sIndR_his_ENSO_related_nor[:,:-1], sIndR_his_ENSO_unrelated_nor[:,:-1]], concat_dim="area")

### sp ssp585

In [15]:
# ssp585_p3
tmp = spssp585_p3_DJF.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
spDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope, spDJF_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")

tmp = spssp585_p3_MAM.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
spMAM_ssp585_p3_north_south_ENSO_related_unrelated_slope, spMAM_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")

tmp = spssp585_p3_JJA.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
spJJA_ssp585_p3_north_south_ENSO_related_unrelated_slope, spJJA_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")

tmp = spssp585_p3_SON.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
spSON_ssp585_p3_north_south_ENSO_related_unrelated_slope, spSON_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, sIndR_ssp585_p3_ENSO_related_nor, sIndR_ssp585_p3_ENSO_unrelated_nor], concat_dim="area")

tmp = spssp585_p3_DJF[:,1:].copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor[:,:-1].time
spnDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope, spnDJF_ssp585_p3_north_south_ENSO_related_unrelated_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor[:,:-1], sIndR_ssp585_p3_ENSO_related_nor[:,:-1], sIndR_ssp585_p3_ENSO_unrelated_nor[:,:-1]], concat_dim="area")

In [16]:
spDJF_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], spDJF_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], spDJF_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=spDJF_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="spDJF partially regression onto north or south ENSO_related or south ENSO_unrelated spDJF"),
)

spMAM_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], spMAM_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], spMAM_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=spMAM_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="spMAM partially regression onto north or south ENSO_related or south ENSO_unrelated spMAM"),
)

spJJA_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], spJJA_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], spJJA_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=spJJA_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="spJJA partially regression onto north or south ENSO_related or south ENSO_unrelated spJJA"),
)

spSON_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], spSON_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], spSON_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=spSON_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="spSON partially regression onto north or south ENSO_related or south ENSO_unrelated spSON"),
)

spnDJF_his_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], spnDJF_his_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], spnDJF_his_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=spnDJF_his_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="spnDJF partially regression onto north or south ENSO_related or south ENSO_unrelated spnDJF"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
spDJF_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spMAM_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spMAM_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
spMAM_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spMAM_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spJJA_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spJJA_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
spJJA_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spJJA_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spSON_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spSON_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
spSON_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spSON_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spnDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spnDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc")
spnDJF_his_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spnDJF_his_north_south_ENSO_related_unrelated_partial_correlation.nc")

In [17]:
spDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], spDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], spDJF_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=spDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="spDJF partially regression onto north or south ENSO_related or south ENSO_unrelated spDJF"),
)

spMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], spMAM_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], spMAM_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=spMAM_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="spMAM partially regression onto north or south ENSO_related or south ENSO_unrelated spMAM"),
)

spJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], spJJA_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], spJJA_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=spJJA_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="spJJA partially regression onto north or south ENSO_related or south ENSO_unrelated spJJA"),
)

spSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], spSON_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], spSON_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=spSON_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="spSON partially regression onto north or south ENSO_related or south ENSO_unrelated spSON"),
)

spnDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], spnDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], spnDJF_ssp585_p3_north_south_ENSO_related_unrelated_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "ENSO_related", "ENSO_unrelated"]),
        models=spnDJF_ssp585_p3_north_south_ENSO_related_unrelated_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="spnDJF partially regression onto north or south ENSO_related or south ENSO_unrelated spnDJF"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
spDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
spMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spMAM_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
spJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spJJA_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
spSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spSON_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spnDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spnDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")
spnDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/spnDJF_ssp585_p3_north_south_ENSO_related_unrelated_partial_correlation.nc")